# CanYouCatchIt?
A web application allowing you to obtain the percentage of chance that your bus/tram/metro is late. 💻🤖🎲🚌 🚎🚇🔮

_Build with the STIB API (available [here](https://opendata.stib-mivb.be/store/))_

# Notes: Making some models 💻🤖🚌 🚎🚇
We are here to make some machine learning models

## Load the data

In [1]:
# Import
import pandas as pd
import os

# Set the path to the directory holding CSV files
DELAY_PATH = '/home/haeresis/Documents/Github/CanYouCatchIt/machine_learning/data'

def load_delay_data(delay_path=DELAY_PATH):
    """
    Load the cvs file in a panda dataframe
    """
    csv_path = os.path.join(delay_path, "delay2020-11-03.csv")
    return pd.read_csv(csv_path)

# load the csv file
delay = load_delay_data()

delay.dropna(inplace=True)
delay.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# Stratifie the data with the different line
# This make sure that the representation of each stop is the same in the train set then in the overall dataset
# This stratification is not necessary is you have enough data
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(delay, delay["stop"]):
    strat_train_set = delay.loc[train_index]
    strat_test_set = delay.loc[test_index]

## Prepare the Data for Machine Learning Algorithms

In [2]:
delay = strat_train_set.drop("delay", axis=1) # drop labels for training set
delay_labels = strat_train_set["delay"].copy()

### Check if there is row with nan

Count the number of row with a nan value

In [3]:
sample_incomplete_rows = delay[delay.isnull().any(axis=1)].head()
len(sample_incomplete_rows.index)

0

### Check if there is attribute that are the same for every row

In [4]:
def unique_cols(df):
    a = df.to_numpy()
    return (a[0] == a).all(0)
unique_cols(delay)

array([ True, False, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False,  True, False,  True])

In [5]:
delay.columns

Index(['transport_type', 'stop', 'line', 'theoretical_time',
       'expectedArrivalTime', 'date', 'direction', 'year', 'month', 'day',
       'hour', 'minute', 'seconds', 'temp', 'humidity', 'visibility', 'wind',
       'rain'],
      dtype='object')

### Data Cleaning

Drop the unnecesery attribute. theoretical_time and expectedArrivalTime are droped because they are string attribute and they are not linked to the delay value. The date attribute is split into hour minute and seconds so it can be droped. The transport_type, year, month, visibility and rain attribute is droped because, it's the same for every row.

In [6]:
nunique = delay.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
delay = delay.drop(cols_to_drop, axis=1)
delay.head()

,stop,line,theoretical_time,expectedArrivalTime,date,direction,hour,minute,seconds,temp,humidity,wind
464,6608G,51,2020-11-03 07:15:09,2020-11-03 07:33:00,2020-11-03 07:11:41.788183,1.0,7,11,41,280.17,93,3.6
3531,6608G,51,2020-11-03 21:37:44,2020-11-03 21:38:00,2020-11-03 21:28:03.371078,1.0,21,28,3,280.64,75,6.2
454,0089,39,2020-11-03 07:11:00,2020-11-03 07:21:00,2020-11-03 07:09:27.615034,0.0,7,9,27,280.17,93,3.6
1580,6608G,51,2020-11-03 12:30:09,2020-11-03 12:22:00,2020-11-03 12:22:03.183467,1.0,12,22,3,284.96,62,6.7
1563,0089,44,2020-11-03 12:18:00,2020-11-03 12:20:00,2020-11-03 12:17:29.258893,0.0,12,17,29,285.07,71,7.7


In [7]:
delay = delay.dropna(subset=["direction"])
delay = delay.drop(["theoretical_time", "expectedArrivalTime", "date"], axis=1)
delay.head()

,stop,line,direction,hour,minute,seconds,temp,humidity,wind
464,6608G,51,1.0,7,11,41,280.17,93,3.6
3531,6608G,51,1.0,21,28,3,280.64,75,6.2
454,0089,39,0.0,7,9,27,280.17,93,3.6
1580,6608G,51,1.0,12,22,3,284.96,62,6.7
1563,0089,44,0.0,12,17,29,285.07,71,7.7


## Handling Text and Categorical Attributes

### Extract the non-numerical attributes

In [8]:
delay_num = delay.drop(['stop'], axis=1)

### Categorical Attributes: The stop and attributes

In [9]:
delay_cat = delay[['stop', 'line']]
delay_cat.head()

,stop,line
464,6608G,51
3531,6608G,51
454,0089,39
1580,6608G,51
1563,0089,44


In [10]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
delay_cat_encoded = ordinal_encoder.fit_transform(delay_cat)
delay_cat_encoded[:10]

array([[1., 2.],
       [1., 2.],
       [0., 0.],
       [1., 2.],
       [0., 1.],
       [0., 0.],
       [1., 2.],
       [0., 1.],
       [1., 3.],
       [1., 2.]])

## Transformation Pipelines



In [11]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np

# get the right column indices: safer than hard-coding indices

def add_extra_features(X, add_combined_time=True):
    hour_ix, minute_ix = [list(delay.columns).index(col) for col in ("hour", "minute")]
    if add_combined_time:
        hour_and_minute = X[:, hour_ix]*3600 + X[:, minute_ix]*60
        return np.c_[X, hour_and_minute]
    else:
        return np.c_[X]

attr_adder = FunctionTransformer(add_extra_features, validate=False,
                                 kw_args={"add_combined_time": True})
delay_extra_attribs = attr_adder.fit_transform(delay.values)

delay_extra_attribs = pd.DataFrame(
    delay_extra_attribs,
    columns=list(delay.columns) + ["hour_and_minute"],
    index=delay.index)
delay_extra_attribs.head()

,stop,line,direction,hour,minute,seconds,temp,humidity,wind,hour_and_minute
464,6608G,51,1,7,11,41,280.17,93,3.6,25860
3531,6608G,51,1,21,28,3,280.64,75,6.2,77280
454,0089,39,0,7,9,27,280.17,93,3.6,25740
1580,6608G,51,1,12,22,3,284.96,62,6.7,44520
1563,0089,44,0,12,17,29,285.07,71,7.7,44220


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_attribs = list(delay_num)
cat_attribs = ["stop", "line"]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler())
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

delay_prepared = full_pipeline.fit_transform(delay)
delay_prepared

array([[-0.17834746,  0.99805888, -1.23164905, ...,  0.        ,
         1.        ,  0.        ],
       [-0.17834746,  0.99805888,  1.45432007, ...,  0.        ,
         1.        ,  0.        ],
       [-0.89779485, -1.0019449 , -1.23164905, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.68022494,  0.99805888, -0.84793918, ...,  0.        ,
         0.        ,  1.        ],
       [-0.17834746,  0.99805888,  0.49504538, ...,  0.        ,
         1.        ,  0.        ],
       [-0.5980251 , -1.0019449 , -0.84793918, ...,  1.        ,
         0.        ,  0.        ]])

## Select and train a model
### Linear Model

In [13]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(delay_prepared, delay_labels)

LinearRegression()

In [14]:
# let's try the full preprocessing pipeline on a few training instances
some_data = delay.iloc[:5]
some_labels = delay_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions: \t", lin_reg.predict(some_data_prepared))
print("Labels: \t", list(some_labels))

Predictions: 	 [-1.9765625  -0.21875    -0.51171875 -3.03125    -2.83203125]
Labels: 	 [-18.0, -1.0, -10.0, 8.0, -2.0]


The model is working although the predictions are not accurate at all

Compute the error

In [15]:
import numpy as np 
from sklearn.metrics import mean_squared_error

delay_predictions = lin_reg.predict(delay_prepared)
lin_mse = mean_squared_error(delay_labels, delay_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

6.09618283489537

Okay, this is better than nothing but clearly not a great score: the delay's value range from 10min to -20min, so a typical prediction error of 6min is not very satisfying.

In [16]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(delay_labels, delay_predictions)
lin_mae

4.1401455735913215

### Decision Tree

In [17]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(delay_prepared, delay_labels)

DecisionTreeRegressor(random_state=42)

In [18]:
delay_predictions = tree_reg.predict(delay_prepared)
tree_mse = mean_squared_error(delay_labels, delay_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

No error? The model has overfit 😖

To evaluate the Decision Tree model we can split the training set into smaller training set and validation set.

### Cross-Validation

The following code performs 10-fold cross-validation

In [19]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores: \t\t", scores)
    print("Mean: \t\t\t", scores.mean())
    print("Standard deviation: \t", scores.std())

#### Linear Regression

In [20]:
lin_scores = cross_val_score(lin_reg, delay_prepared, delay_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: 		 [5.98857984 5.91333928 5.73350558 6.10935796 6.02476883 6.46016862
 6.26279492 6.03696304 5.9969698  6.59106597]
Mean: 			 6.111751382143091
Standard deviation: 	 0.24451983060456026


#### Decision Tree

In [21]:
scores = cross_val_score(tree_reg, delay_prepared, delay_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

Scores: 		 [6.66294394 6.52379265 5.96320975 5.958595   5.9078687  6.04834889
 6.24396148 6.05984182 6.8186147  6.89908708]
Mean: 			 6.308626402943865
Standard deviation: 	 0.3628815791293764


We can see taht the Decision Tree model is overfitting so strongly that the Linear Regression model performs even better.

#### Random Forest

In [22]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(delay_prepared, delay_labels)

RandomForestRegressor(n_estimators=10, random_state=42)

In [23]:
delay_predictions = forest_reg.predict(delay_prepared)
forest_mse = mean_squared_error(delay_labels, delay_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

2.1694736533644483

In [24]:
forest_scores = cross_val_score(forest_reg, delay_prepared, delay_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: 		 [4.74573561 5.07600804 4.78116247 4.73061023 5.17840612 4.92521089
 5.07895907 4.88931203 5.43763341 5.77118627]
Mean: 			 5.061422413828156
Standard deviation: 	 0.31548707800416


With the Random Forest we have a mutch better results. But we see a significant difference between the score on the training set than on the validation set. That means that the model is still overfitting the training set.

#### Support Vector Regression

In [25]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(delay_prepared, delay_labels)

SVR(kernel='linear')

In [26]:
delay_predictions = svm_reg.predict(delay_prepared)
svm_mse = mean_squared_error(delay_labels, delay_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

6.205347072789438

In [27]:
svr_scores = cross_val_score(svm_reg, delay_prepared, delay_labels, scoring="neg_mean_squared_error", cv=10)
svr_rmse_scores = np.sqrt(-svr_scores)
display_scores(svr_rmse_scores)

Scores: 		 [6.07844444 6.02334214 5.73308446 6.2357253  6.03127925 6.60971848
 6.46658438 6.0476503  6.11625581 6.70950142]
Mean: 			 6.205158598222868
Standard deviation: 	 0.28674421109776305


We can see that the best model is the Random Forest 🌲

## Fine-Tune the models
### Grid Search

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(delay_prepared, delay_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [29]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 3, 'n_estimators': 10}

In [30]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features=3, n_estimators=10,
                      random_state=42)

In [31]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

5.229293888066451 {'max_features': 2, 'n_estimators': 3}
4.659063141262619 {'max_features': 2, 'n_estimators': 10}
4.4734422992682745 {'max_features': 2, 'n_estimators': 30}
5.201049487437042 {'max_features': 4, 'n_estimators': 3}
4.698066307116477 {'max_features': 4, 'n_estimators': 10}
4.559904164720108 {'max_features': 4, 'n_estimators': 30}
5.60837167289671 {'max_features': 6, 'n_estimators': 3}
4.9453556101190435 {'max_features': 6, 'n_estimators': 10}
4.688523607161991 {'max_features': 6, 'n_estimators': 30}
5.638335524725624 {'max_features': 8, 'n_estimators': 3}
4.948178061096842 {'max_features': 8, 'n_estimators': 10}
4.78635040830151 {'max_features': 8, 'n_estimators': 30}
4.983790765404622 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
4.559545381011011 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
4.835647494921637 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
4.462856505830779 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10

In [32]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.024538,0.001180,0.003820,0.000967,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-24.963862,-25.140777,...,-27.345515,4.313622,16,-7.079622,-6.797886,-7.338012,-6.677279,-6.921085,-6.962777,0.230170
1,0.053706,0.013845,0.005456,0.000914,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-20.435712,-19.806748,...,-21.706869,2.717804,6,-4.217870,-3.992247,-4.316769,-3.901218,-4.078300,-4.101281,0.149980
2,0.132541,0.001953,0.009894,0.001415,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-19.074712,-18.552282,...,-20.011686,2.246948,2,-3.280728,-3.348030,-3.376709,-3.161179,-3.179419,-3.269213,0.086758
3,0.018125,0.000368,0.003302,0.000683,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-25.644013,-28.609313,...,-27.050916,1.205006,15,-8.393342,-7.766757,-7.949123,-7.435451,-7.267908,-7.762516,0.396078
4,0.054444,0.000843,0.003948,0.001099,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-20.778414,-21.214822,...,-22.071827,1.540632,8,-4.282619,-4.040830,-4.453498,-4.317390,-4.154690,-4.249805,0.141317
5,0.162517,0.003371,0.009972,0.000605,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-19.263017,-19.239583,...,-20.792726,2.059615,4,-3.323520,-3.429246,-3.526319,-3.303476,-3.328401,-3.382192,0.084284
6,0.022500,0.001080,0.003271,0.000749,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-31.412621,-27.581266,...,-31.453833,2.494298,17,-9.691813,-8.031669,-8.497661,-8.141193,-8.495301,-8.571527,0.590414
7,0.065054,0.001794,0.003921,0.000941,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-23.823204,-21.652848,...,-24.456542,2.604164,11,-4.716113,-4.455692,-4.695599,-4.566236,-4.558600,-4.598448,0.096217
8,0.194947,0.005527,0.009302,0.000952,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-21.753017,-19.596897,...,-21.982254,2.378525,7,-3.759619,-3.531016,-3.679879,-3.470677,-3.536216,-3.595481,0.107060
9,0.027154,0.001367,0.002455,0.000559,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-35.320568,-29.426286,...,-31.790827,3.273996,18,-8.383311,-9.154521,-8.644220,-8.044337,-8.241018,-8.493481,0.384006


We should evaluate higher values of n_estimators as well, since 30 is the maximum value that was evaluated. The score may continue to improve.

In [38]:
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [30, 40, 50], 'max_features': [1, 2, 4, 6]},
    # then try 9 (3×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [70, 80, 90], 'max_features': [1, 2, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+9)*5=105 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(delay_prepared, delay_labels)
print(grid_search.best_params_, "\n")
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

{'bootstrap': False, 'max_features': 1, 'n_estimators': 80} 

4.540977874804955 {'max_features': 1, 'n_estimators': 30}
4.562865097664219 {'max_features': 1, 'n_estimators': 40}
4.5524070620829615 {'max_features': 1, 'n_estimators': 50}
4.4734422992682745 {'max_features': 2, 'n_estimators': 30}
4.479920765357988 {'max_features': 2, 'n_estimators': 40}
4.463736841984938 {'max_features': 2, 'n_estimators': 50}
4.559904164720108 {'max_features': 4, 'n_estimators': 30}
4.539904099741907 {'max_features': 4, 'n_estimators': 40}
4.523608057770053 {'max_features': 4, 'n_estimators': 50}
4.688523607161991 {'max_features': 6, 'n_estimators': 30}
4.661594051054611 {'max_features': 6, 'n_estimators': 40}
4.648850702229582 {'max_features': 6, 'n_estimators': 50}
4.3411853613431965 {'bootstrap': False, 'max_features': 1, 'n_estimators': 70}
4.329743570952857 {'bootstrap': False, 'max_features': 1, 'n_estimators': 80}
4.341578002930521 {'bootstrap': False, 'max_features': 1, 'n_estimators': 90}
4.337

### Randomized Search

In [34]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(delay_prepared, delay_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc7c2349a30>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc7c2349700>},
                   random_state=42, scoring='neg_mean_squared_error')

In [35]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

4.64679684407939 {'max_features': 7, 'n_estimators': 180}
4.645046895105344 {'max_features': 5, 'n_estimators': 15}
4.483782458358234 {'max_features': 3, 'n_estimators': 72}
4.603851540331541 {'max_features': 5, 'n_estimators': 21}
4.662321696765109 {'max_features': 7, 'n_estimators': 122}
4.478646745083121 {'max_features': 3, 'n_estimators': 75}
4.474124792618715 {'max_features': 3, 'n_estimators': 88}
4.521003573113413 {'max_features': 5, 'n_estimators': 100}
4.461928090323428 {'max_features': 3, 'n_estimators': 150}
5.732414844911135 {'max_features': 5, 'n_estimators': 2}


### Analyze the Best Models and Their Errors

In [36]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([0.01308774, 0.00258553, 0.10606256, 0.1867887 , 0.13633706,
       0.1603047 , 0.06165768, 0.09625416, 0.2052632 , 0.00224472,
       0.00365733, 0.00552693, 0.00812962, 0.00702122, 0.00507886])

In [37]:
extra_attribs = ["hour_and_minute"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.20526319552474567, 'hour_and_minute'),
 (0.18678870350688273, 'minute'),
 (0.16030470129491844, 'temp'),
 (0.13633705794631232, 'seconds'),
 (0.10606256140783749, 'hour'),
 (0.09625416145659321, 'wind'),
 (0.061657681740862245, 'humidity'),
 (0.013087736041830202, 'line'),
 (0.0036573313968327726, '6608G'),
 (0.002585526109623008, 'direction'),
 (0.002244716316602857, '0089')]

We can see that the most important value is the newly added "hour_and_minute" attribute.
### Analyze the Best Models and Their Errors

In [41]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("delay", axis=1)
y_test = strat_test_set["delay"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

4.220043097546371